In [4]:
import os
import re
os.environ['OPENAI_API_KEY'] = ''

### Multi-Agent Debate
Debate_Agents：透過兩個智能體的辯論，優化清單中的指定元件

In [2]:
from agent import Agent
    
class Debate_Agents(Agent):
    
    def __init__(self, agent_name, product, first_company, second_company, model, limited_time=2):
        
        self.first_company = first_company
        self.second_company = second_company
        self.product = product
        
        prompt = """
            你現在是{A_company}的廠商代表，對於自己生產的產品瞭若指掌，且非常有自信。
            你的對面坐著{B_company}，他是另一個{product}的廠商代表。
            你們正在進行著挑選哪一個{product}的辯論。
            
            現在，請根據user的問題，試著去說服user採用你們家廠商生產的產品，
            說服他你的觀點才是更好的。
        """
        
        rules_prompt = """
            下面有三個規則要遵守：
            1. 字數內容100字為上限
            2.提出產品名稱
            3.提出自身產品優勢內容
            4.提出另一牌子的缺點
        """
        
        first_prompt = prompt.format(A_company=self.first_company, 
                                     B_company=self.second_company, 
                                     product=self.product) + """
            用戶需求為：{topic}
            {Opposition}廠商的發言：{previous_message}
        """ + rules_prompt
        
        self.first_agent = Agent(self.first_company, first_prompt, model)
        
        second_prompt = prompt.format(A_company=self.second_company, 
                                     B_company=self.first_company, 
                                     product=self.product) + """
            用戶需求為：{topic}
            {Opposition}廠商的發言：{previous_message}
        """ + rules_prompt
        
        self.second_agent = Agent(self.second_company, second_prompt, model)
        

        prompt = f"""
                你是一個專業的{self.product}專家，現在，有兩個廠商代表著自己家的產品正在進行辯論。
                目前有：{self.first_company}的代表與{self.second_company}的代表。\n
        """
        
        summary_prompt = prompt + """
                以下為他們的爭辯過程：
                {memory}

                請根據他們的爭辯內容，根據客戶的需求：{topic}，
                選出你覺得最合適客戶需求的廠商與它推薦的產品，並預估該產品的價格。

                輸出範例：
                [ 推薦原因 ] ： 耐用性強，可以使用到最新規格的Intel CPU...
                [ 產品名稱 ] ： Inter 第八代 i9
                [ 產品價格 ] ： 13000

                另外有幾點請注意：
                1. 推薦原因的說明請限制在50個字以內
                2. 產品的價格，請回覆一個完整的數字，中間不需要加入逗號
        """
        
        super().__init__(agent_name, summary_prompt, model)
        
        self.limited_time = limited_time

    def forward(self, component_dict):

        previous_message = ''
        time = 1
        
        question = f'我目前預算在{component_dict[self.product]["price"]}的附近，想要買一個{self.product}，請幫我推薦'

        while self.limited_time != time:

            round_message = f' ======= 第 {time} 回合 ======= \n題目：{question}'

            first_response = self.first_agent.llm.invoke({"topic":question,
                                                      "previous_message":previous_message,
                                                      "Opposition":self.second_agent})
            previous_message = f'{self.first_agent.agent_name}廠商的發言：{first_response}\n'
            self.memory.append(previous_message)

            second_response = self.second_agent.llm.invoke({"topic":question,
                                                        "previous_message":previous_message,
                                                        "Opposition":self.first_agent})
            previous_message = f'{self.second_agent.agent_name}廠商的發言：{second_response}\n'
            self.memory.append(previous_message)

            time += 1
        
        memory_string = ''.join(self.memory)

        recommendation = self.llm.invoke({"topic":question, "memory":memory_string})
        
        component_dict[self.product] = self.parser_message(recommendation)
        
        return component_dict
        
    def parser_message(self, message):
        pattern = {
            "reason": r"\[ 推薦原因 \] ：\s*(.*?)\s*\n",
            "name": r"\[ 產品名稱 \] ：\s*(.*?)\s*\n",
            "price": r"\[ 產品價格 \] ：\s*(\d+)"
        }

        results = {key: re.search(regex, message).group(1) for key, regex in pattern.items()}

        return results

In [3]:
# 測試清單
component_dict = {
    'CPU':{   
        'reason': '性能優越，適合遊戲和多任務處理，性價比高。',
        'name': 'AMD Ryzen 7 5800X',
        'price': '10900'
        },
    'GPU':{
        'reason': '性能優越，適合高效能遊戲和專業應用，散熱效能佳，耐用性高。',
        'name': '微星 GeForce RTX 3060 Ti',
        'price': '15900'
        }
    }

In [4]:
debate_cpu_agents = Debate_Agents('debate', 'CPU', 'Intel', 'AMD', 'gpt-4o-mini', 2)
debate_gpu_agents = Debate_Agents('debate', 'GPU', '微星', '技嘉', 'gpt-4o-mini', 2)

In [5]:
component_dict = debate_cpu_agents(component_dict)
component_dict = debate_gpu_agents(component_dict)

print(f'新組合清單：')
component_dict

新組合清單：


{'CPU': {'reason': '性價比高，能效優秀，適合遊戲和多任務處理。',
  'name': 'AMD Ryzen 5 5600X',
  'price': '9000'},
 'GPU': {'reason': '微星的RTX 3060 Ti提供出色性能與先進散熱，適合長時間遊戲使用。',
  'name': '微星 RTX 3060 Ti',
  'price': '15900'}}

### Multi-Agent Debate System
- Debate_Agents：透過兩個智能體的辯論，優化清單中的指定元件
- Planning_Agent：根據使用者的需求，分配預算給所有的元件
- Multi_Agent_Debate_System：建立八個元件的組合，每個組合兩個agents，共同從既有預算規劃出一個完整的電腦清單

In [5]:
from units import get_budget_from_string, print_results
from multi_agent import PC_DIY_Debate_System

model_name = 'gpt-4o-mini'
require = '有一個使用者想要50000元的遊戲機器'
budget = get_budget_from_string(require)

pc_diy = PC_DIY_Debate_System(model_name,require , budget)

component_dict = pc_diy.execute({})
print_results(component_dict)

組合清單：
1. Mother board: 華碩 TUF Gaming B550M-PLUS | price: 3000
2. Case: 全漢 H500 | price: 1500
3. CPU: AMD Ryzen 5 5600X | price: 9000
4. GPU: 微星 GeForce RTX 3060 Ti | price: 14500
5. Memory: 金士頓 HyperX Fury | price: 5800
6. Storage: 威剛 S10 SSD | price: 3000
7. Power: 華碩 Power Pro 2000 | price: 1999
8. Fan: 威剛 XFan 120 | price: 500

總價格：39299


### Test dataset

In [6]:
questions = [
    '請問30000元內能組一台什麼樣的主機，主要是玩LOL和看Netflix用？',
    '有推薦的組合嗎？預算大概在50000元，要能處理攝影後製和輕遊戲，如：軒轅劍7。',
    '碩士生研究用的電腦，預算約70000元，需要跑深度學習模型，有沒有建議配置？',
    '預算大約20000元，尋求一台能寫報告和上網的簡易電腦，要求瀏覽器多開流暢。',
    '想請問80000元左右，能組出來什麼級別的遊戲主機？想玩些新遊戲，畫質要好。',
    '工作需要，我要一台能跑Solidworks和AutoCAD的工作站，預算大概60000元，有建議嗎？',
    '有沒有推薦的遊戲主機配置？預算在45000元，想要玩 POE、APEX等槍戰、LOL。',
    '預算約45000元，需要一台主要用來跑Adobe系列軟體的電腦，特別是Photoshop和After Effects。',
    '我需要一台適合股市分析的電腦，預算大概30000元，需要大量運算和多螢幕，有什麼好提議？',
    '想要組一台約65000元的電腦，主要用於VR遊戲和一般辦公，有沒有推薦配置？',
    '請問40000元預算可以組出來能玩FPS、3A大作、多開模擬器遊戲的電腦嗎？',
    '預算25000元，我想要一台電腦主要用於線上學習和偶爾寫程式，有建議配置嗎？',
    '請問75000元左右的預算能組一台什麼樣的4K影片剪輯機？',
    '我要組一台電腦，預算約35000元，主要用於玩《英雄聯盟》和做寫作業與論文研究，推薦配置是？',
    '預算大概55000元，尋找一台適合做3D動畫和模型設計的電腦，有推薦嗎？',
    '我是遊戲開發學生，預算大約85000元，需要一台能處理遊戲開發和渲染的高階機，有沒有好的配置？',
    '預算20000元內，請問有適合辦公和輕度Photoshop使用的電腦配置嗎？',
    '想問下，有沒有大約50000元的電腦配置，適合玩大型開放世界遊戲的？',
    '我需要一台電腦，預算大概45000元，主要用於音樂製作和錄音，需要特別注意什麼配件？',
    '預算約60000元，想要一台電腦主要用於科學計算和寫程式，有推薦配置嗎？',
    '請問我想組一台約70000元的電腦，主要用於虛擬機運行和大數據處理，有什麼建議？',
    '預算大約30000元，尋找一台適合初學者學習寫程式和偶爾玩遊戲的電腦，有推薦配置嗎？',
    '我要組裝一台預算約40000元的電腦，主要用來看4K電影和日常使用，有建議配置嗎？',
    '想請問，85000元預算能組一台什麼樣的專業級遊戲機？希望能玩最新遊戲。',
    '預算約50000元，我想要一台電腦主要用於AI學習和研究，有沒有好的配置建議？',
    '請問一下，25000元預算可以組一台適合高中生用的課業學習電腦嗎？主要是寫作業和上網。',
    '我是平面設計師，預算大概55000元，需要一台適合設計工作的電腦，有推薦嗎？',
    '預算大約70000元，想要一台適合專業影像處理和影片剪輯的電腦，有什麼建議配置？',
    '我想組一台電腦，預算約60000元，主要用於玩模擬飛行遊戲，有沒有推薦的配置？',
    '請問有沒有大約35000元的電腦配置推薦，適合用於網頁開發和輕度遊戲？'
]

In [19]:
questions = [
    # AutoCAD
    '預算約50000元，想要一台能高效運行AutoCAD進行工程繪圖與設計的電腦，特別需要多核心CPU來加速運算。',
    # Photoshop
    '預算大概35000元，需要一台能流暢執行Photoshop圖像編輯的電腦，GPU加速很重要。',
    # Illustrator
    '預算約30000元，希望有台能穩定跑Illustrator的電腦，CPU和內存要足夠強大，處理大型向量檔不會卡。',
    # Premiere Pro
    '我有約60000元預算，希望組一台能流暢剪接4K影片的電腦，用Premiere Pro時需強大GPU和大量記憶體。',
    # After Effects
    '預算約65000元，想要一台能順暢處理After Effects動態特效的電腦，GPU和大容量內存非常關鍵。',
    # Blender
    '預算50000元左右，需一台適合Blender 3D建模和GPU渲染的電腦，能加速NVIDIA CUDA運算。',
    # 3ds Max
    '預算70000元，想組一台能高效運行3ds Max的工作站，要求對GPU和CPU性能都有極高規格。',
    # Maya
    '大約75000元預算，需要一台能在Maya中順暢做3D建模和專業動畫渲染的電腦。',
    # ZBrush
    '預算約45000元，需要一台能流暢使用ZBrush進行數位雕刻的電腦，要求大容量內存和高速儲存。',
    # SolidWorks
    '預算約60000元，想要一台能跑SolidWorks工程設計軟體的工作站，多核心CPU和中高階GPU不可少。',
    # Revit
    '有大約55000元預算，想買台能穩定跑Revit做建築設計與BIM建模的電腦，需要充裕內存和不錯的GPU。',
    # DaVinci Resolve
    '預算60000元左右，要一台能在DaVinci Resolve中流暢剪輯與調色的電腦，GPU加速效果很重要。',
    # Vegas Pro
    '預算50000元左右，需要能跑Vegas Pro的電腦，多核CPU和高效能GPU是必須，以加快影片編輯。',
    # Cinema 4D
    '預算約65000元，希望組一台適合Cinema 4D動畫與渲染的電腦，GPU和內存絕不能馬虎。',
    # MATLAB
    '預算大約40000元，想組一台能執行MATLAB科學計算的電腦，多核心處理器與大內存很關鍵。',
    # Unreal Engine
    '預算80000元，我需要一台能順暢跑Unreal Engine的開發機，高效能GPU和多核CPU都要有。',
    # Cyberpunk 2077
    '預算約50000元，希望有一台能在高畫質下跑Cyberpunk 2077的電腦，需要強悍的GPU（如RTX系列）。',
    # Red Dead Redemption 2
    '預算60000元左右，想組一台流暢運行Red Dead Redemption 2的電腦，GPU、CPU和內存都要高規。',
    # Grand Theft Auto V (GTA V)
    '預算約40000元，我想要一台能長期流暢玩GTA V的主機，硬體規格要足以應對大型開放世界遊戲。',
    # Call of Duty: Modern Warfare II
    '預算45000元，需要一台能在高畫質下跑Modern Warfare II的電腦，GPU性能非常重要。',
    # The Witcher 3
    '預算35000元，想組一台能高品質運行The Witcher 3的電腦，GPU和內存要足夠充裕。',
    # Elden Ring
    '預算40000元左右，需要一台可流暢跑Elden Ring的電腦，強大的GPU與多核心CPU很重要。',
    # Dota 2
    '預算25000元，希望有台能流暢玩Dota 2的電腦，CPU和GPU配置需均衡。',
    # Counter-Strike 2
    '預算30000元，要一台適合玩CS2的電腦，最好能搭配高刷新率螢幕並有足夠GPU性能。',
    # Unity
    '預算約50000元，需要一台能跑Unity遊戲開發的電腦，要有足以支撐即時3D預覽的強勁GPU。',
    # Visual Studio
    '預算約30000元，希望組一台能穩定執行Visual Studio開發環境的電腦，多核心CPU很有幫助。',
    # Eclipse
    '預算25000元，需要一台能流暢跑Eclipse做軟體開發的電腦，高速儲存和多核CPU不可或缺。',
    # Android Studio
    '預算35000元，需要一台能快速執行Android Studio的電腦，有足夠內存和不錯CPU性能。',
    # Docker
    '預算45000元，希望組一台適合Docker多環境執行的電腦，需要大內存和高速CPU來同時跑多個容器。',
    # VMware Workstation
    '預算50000元，需要一台能流暢執行VMware Workstation虛擬化的電腦，內存和CPU虛擬化能力需很強。'
]


In [20]:
from units import get_budget_from_string, print_results
from multi_agent import PC_DIY_Debate_System

result_data = {}

for i, question in enumerate(questions):

    print(f'第{i}筆資料：{question}')
    
    require = question
    budget = get_budget_from_string(require)
        
    model_name = 'gpt-4o-mini'
    budget = get_budget_from_string(require)

    pc_diy = PC_DIY_Debate_System(model_name, require , budget)

    require_done = True
    
    while require_done:
        try:
            component_dict = pc_diy.execute({})
            require_done = False
        except Exception:
            print('Error')
            
    # component_dict = pc_diy.execute({})
            
    result_data[require] = component_dict

第0筆資料：預算約50000元，想要一台能高效運行AutoCAD進行工程繪圖與設計的電腦，特別需要多核心CPU來加速運算。
第1筆資料：預算大概35000元，需要一台能流暢執行Photoshop圖像編輯的電腦，GPU加速很重要。
第2筆資料：預算約30000元，希望有台能穩定跑Illustrator的電腦，CPU和內存要足夠強大，處理大型向量檔不會卡。
第3筆資料：我有約60000元預算，希望組一台能流暢剪接4K影片的電腦，用Premiere Pro時需強大GPU和大量記憶體。
第4筆資料：預算約65000元，想要一台能順暢處理After Effects動態特效的電腦，GPU和大容量內存非常關鍵。
第5筆資料：預算50000元左右，需一台適合Blender 3D建模和GPU渲染的電腦，能加速NVIDIA CUDA運算。
第6筆資料：預算70000元，想組一台能高效運行3ds Max的工作站，要求對GPU和CPU性能都有極高規格。
第7筆資料：大約75000元預算，需要一台能在Maya中順暢做3D建模和專業動畫渲染的電腦。
第8筆資料：預算約45000元，需要一台能流暢使用ZBrush進行數位雕刻的電腦，要求大容量內存和高速儲存。
第9筆資料：預算約60000元，想要一台能跑SolidWorks工程設計軟體的工作站，多核心CPU和中高階GPU不可少。
第10筆資料：有大約55000元預算，想買台能穩定跑Revit做建築設計與BIM建模的電腦，需要充裕內存和不錯的GPU。
第11筆資料：預算60000元左右，要一台能在DaVinci Resolve中流暢剪輯與調色的電腦，GPU加速效果很重要。
第12筆資料：預算50000元左右，需要能跑Vegas Pro的電腦，多核CPU和高效能GPU是必須，以加快影片編輯。
第13筆資料：預算約65000元，希望組一台適合Cinema 4D動畫與渲染的電腦，GPU和內存絕不能馬虎。
第14筆資料：預算大約40000元，想組一台能執行MATLAB科學計算的電腦，多核心處理器與大內存很關鍵。
第15筆資料：預算80000元，我需要一台能順暢跑Unreal Engine的開發機，高效能GPU和多核CPU都要有。
第16筆資料：預算約50000元，希望有一台能在高畫質下跑Cyberpunk 2077的電腦，需要強悍的GPU（如RTX系列）。

In [21]:
import json

# 將資料存放到 questions.json 檔案
with open('debate_results.json', 'w', encoding='utf-8') as file:
    json.dump(result_data, file, ensure_ascii=False, indent=4)

print("資料已成功儲存到 questions.json！")

資料已成功儲存到 questions.json！


In [18]:
result_data

{'請問30000元內能組一台什麼樣的主機，主要是玩LOL和看Netflix用？': {'Mother board': {'reason': '華碩 TUF Gaming B550-PLUS 擁有優秀的耐用性與穩定性，適合長時間運行。',
   'name': '華碩 TUF Gaming B550-PLUS',
   'price': '2999'},
  'Case': {'reason': '酷碼X1具優異散熱性能及擴展性，設計優雅，耐用且易於安裝，適合您的需求。',
   'name': '酷碼X1',
   'price': '1999'},
  'CPU': {'reason': '多核心性能優秀，性價比高，適合多任務和創作工作。',
   'name': 'AMD Ryzen 5 5600X',
   'price': '10500'},
  'GPU': {'reason': '出色的性能和散熱設計，適合1080p和1440p遊戲，且具備良好的客戶服務保障。',
   'name': '微星 GeForce RTX 3060 Ti',
   'price': '15000'},
  'Memory': {'reason': '美光的Crucial Ballistix 16GB DDR4性能穩定，適合預算6000元的客戶需求。',
   'name': '美光 Crucial Ballistix 16GB DDR4',
   'price': '5500'},
  'Storage': {'reason': '威剛XPG GAMMIX S70速度更快，適合遊戲與大數據處理，性價比高。',
   'name': '威剛 XPG GAMMIX S70 NVMe SSD',
   'price': '3800'},
  'Power': {'reason': '高性價比，具備過熱保護，設計便於攜帶，適合多種場合使用。',
   'name': '海韻 Power Pro 3000',
   'price': '2999'},
  'Fan': {'reason': '靜音設計與高效散熱，適合安靜環境下使用。',
   'name': '威剛 X-Fan 120',
   'price': '900'}},
 '有推薦的組合嗎？預算大概在50000元，要能處理